In [1]:
from netCDF4 import Dataset
import numpy as np
import dateutil.parser
import matplotlib.pyplot as plt
import os
from glob import glob
import tqdm
import pandas as pd
import datetime
from metpy.calc import add_pressure_to_height
from metpy.calc import sigma_to_pressure
import warnings
import pickle
from metpy.units import units

In [2]:
path = '/home/robbie/Dropbox/BLWG_Obs/data/SHEBA/rawinsonde.nc'
path = '../data/SHEBA/rawinsonde.nc'
d = Dataset(path)
d

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    description: SHEBA rawinsonde data
    author: Stephan de Roode
    date: 7 June 2000
    dimensions(sizes): time(816), levels(179)
    variables(dimensions): float64 time(time), float64 surf_pres_sonde(time), float64 surf_temp(time), float64 surf_wtot(time), float64 surf_wdir(time), float64 surf_pres_tower(time), float64 sigma(levels), float64 temp(time, levels), float64 rh(time, levels), float64 wtot(time, levels), float64 wdir(time, levels)
    groups: 

In [3]:
d['time'][:]

masked_array(data=[289.16666667, 290.83333333, 290.95833333, 292.16666667,
                   292.95833333, 293.45833333, 293.95833333, 294.45833333,
                   294.95833333, 295.45833333, 295.95833333, 296.45833333,
                   296.95833333, 297.45833333, 297.95833333, 298.45833333,
                   298.95833333, 299.95833333, 300.45833333, 300.95833333,
                   301.45833333, 301.95833333, 302.45833333, 302.95833333,
                   303.95833333, 304.58333333, 305.45833333, 305.95833333,
                   306.5       , 306.625     , 306.95833333, 307.45833333,
                   307.95833333, 308.95833333, 310.        , 310.45833333,
                   310.95833333, 311.08333333, 311.45833333, 312.        ,
                   312.45833333, 312.625     , 312.95833333, 313.45833333,
                   313.95833333, 314.45833333, 314.95833333, 315.45833333,
                   315.95833333, 316.45833333, 316.95833333, 317.5       ,
                   317.95

In [4]:
sigmas = np.array(d['sigma'])

pressures = []
heights = []

for p0 in tqdm.tqdm(d['surf_pres_sonde']):
    
    pressure_profile = sigma_to_pressure(sigmas,
                                         float(p0)* units.hPa,
                                         0* units.hPa).magnitude
    
    height_profile = [add_pressure_to_height(0 * units.meters,
                                             (p0-pp) * units.hPa).magnitude for pp in pressure_profile]
    
    pressures.append(pressure_profile)
    
    heights.append(height_profile)

    
pressures = np.asarray(pressures)
heights = np.asarray(heights)*1000

 18%|███████▎                                 | 145/816 [00:16<01:14,  9.03it/s]/Users/ali/opt/anaconda3/lib/python3.9/site-packages/pint/facets/plain/quantity.py:1374: RuntimeWarning: invalid value encountered in double_scalars
  magnitude = new_self._magnitude**exponent
100%|█████████████████████████████████████████| 816/816 [01:30<00:00,  8.97it/s]


In [5]:
pickle.dump((pressures,heights),open('sheba_p_h.p','wb'))

In [6]:
dt0 = datetime.datetime(1997,1,1)

dates = [datetime.timedelta(days=int(np.floor(x))) + dt0 for x in d['time']]

hours = [datetime.timedelta(hours=(x%1)*24) for x in d['time']]

dts = [date + hour for (date, hour) in zip(dates, hours)]

In [7]:
outs = []
for i in tqdm.trange(d['time'].shape[0]):
    outs.append(dts[i].month)

print(set(outs))

100%|█████████████████████████████████████| 816/816 [00:00<00:00, 457927.76it/s]

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}


In [8]:
warnings.filterwarnings("ignore")

In [9]:
list_of_dicts = []

for i in tqdm.trange(d['time'].shape[0]):
    
    dic = {'temp':d['temp'][i], 'pres': pressures[i], 'height': heights[i], 'rh' : d['rh'][i], 'wind':d['wtot'][i]}
    
    df = pd.DataFrame(dic)
    
    df = df[df['temp']>-200]
    df = df[df['temp']<15]
    
    temp = np.asarray(df['temp'])
    alt = np.asarray(df['height'])
    press = np.asarray(df['pres'])
    wind = np.asarray(df['wind'])
    
    t0 = np.nanmedian(temp[(alt<10) & (alt>-10)])
    inversion_height = alt[np.argmax(temp)]
    inversion_strength = np.max(temp) - t0
    
    t850hpa = temp[np.argmin(np.abs(press-850))]
    t2m = temp[np.argmin(np.abs(alt-2))]
    lls = t850hpa - t2m
    
    rh_sub_850 = np.nanmean(df[df['pres']<851]['rh'])
    
    
    w0 = float(d['surf_wtot'][i])
    w850 = wind[np.argmin(np.abs(press-850))]
    
    dic = {'t0':t0,
       'dt0':dts[i],
        'month':dts[i].month,
       'inversion_strength':inversion_strength,
       'inversion_height':inversion_height,
        'low_level_stability':lls,
        'rh_sub_850':rh_sub_850,
       'surf_wind_velocity':w0,
       'wind_shear':w850-w0,
      }

    list_of_dicts.append(dic)
    
#     break


df_ = pd.DataFrame(list_of_dicts)


100%|████████████████████████████████████████| 816/816 [00:00<00:00, 862.89it/s]


In [10]:
df_c = df_.dropna()

df_c['dt_'] = [x.to_datetime64() for x in df_c['dt0']]

# df_c = df_c[df_c['inversion_height'] < 10,000]

df_c = df_c[np.isin(df_c['month'], [9,10,11,12,1,2,3])]

In [11]:
df_c.to_csv('../data/tables/SHEBA.csv')

In [12]:
df_c

,t0,dt0,month,inversion_strength,inversion_height,low_level_stability,rh_sub_850,surf_wind_velocity,wind_shear,dt_
0,-16.225000,1997-10-17 04:00:00,10,0.000000,-0.000000,-3.641667,0.576609,-999.0,1005.283251,1997-10-17 04:00:00
1,-17.854545,1997-10-18 20:00:00,10,3.654545,877.984821,1.354545,0.602644,-999.0,1001.289822,1997-10-18 20:00:00
2,-17.950000,1997-10-18 23:00:00,10,3.950000,619.460308,1.750000,0.600578,-999.0,1001.906664,1997-10-18 23:00:00
3,-13.225000,1997-10-20 04:00:00,10,0.625000,931.700101,-2.175000,0.444664,-999.0,1004.970000,1997-10-20 04:00:00
4,-8.026923,1997-10-20 23:00:00,10,0.000000,-0.000000,-6.073077,0.887422,-999.0,1010.645000,1997-10-20 23:00:00
...,...,...,...,...,...,...,...,...,...,...
811,-0.878788,1998-10-07 23:00:00,10,0.000000,-0.000000,-11.204545,0.731400,-999.0,1013.581602,1998-10-07 23:00:00
812,-18.361111,1998-10-08 23:00:00,10,10.961111,734.584421,8.794444,0.455908,-999.0,1004.046662,1998-10-08 23:00:00
813,0.228000,1998-10-09 11:00:00,10,0.000000,-0.000000,-9.248000,0.828475,-999.0,1015.959831,1998-10-09 11:00:00
814,9.071429,1998-10-09 23:00:00,10,0.000000,-0.000000,-15.338095,0.839477,-999.0,1015.753316,1998-10-09 23:00:00


In [13]:
df_c.sort_values('dt0')[200:250]

,t0,dt0,month,inversion_strength,inversion_height,low_level_stability,rh_sub_850,surf_wind_velocity,wind_shear,dt_
210,-18.866667,1998-02-02 11:00:00,2,2.833333,2201.737795,0.716667,0.507775,-999.00,1006.774855,1998-02-02 11:00:00
211,-19.847619,1998-02-02 23:00:00,2,2.080952,1918.142471,1.897619,0.714014,-999.00,1007.334844,1998-02-02 23:00:00
212,-29.500000,1998-02-03 12:00:00,2,12.650000,865.474114,11.433333,0.694217,-999.00,1005.185946,1998-02-03 12:00:00
213,-32.609091,1998-02-03 23:00:00,2,15.959091,777.220426,14.609091,0.564227,-999.00,1004.219927,1998-02-03 23:00:00
214,-33.600000,1998-02-04 12:00:00,2,17.300000,1004.123327,16.150000,0.507104,-999.00,0.000000,1998-02-04 12:00:00
215,-32.045000,1998-02-04 23:00:00,2,15.795000,1849.646757,14.345000,0.387284,-999.00,1010.006175,1998-02-04 23:00:00
216,-28.177778,1998-02-05 11:00:00,2,13.577778,2056.334075,12.344444,0.280976,-999.00,0.000000,1998-02-05 11:00:00
217,-16.636364,1998-02-06 23:00:00,2,7.136364,1344.580732,6.696364,0.782622,-999.00,1005.939888,1998-02-06 23:00:00
218,-18.075862,1998-02-07 11:00:00,2,6.325862,1576.187462,5.475862,0.733157,-999.00,1012.275000,1998-02-07 11:00:00
219,-33.187500,1998-02-07 23:00:00,2,8.287500,2299.049525,7.587500,0.248435,-999.00,1017.559322,1998-02-07 23:00:00
